In [1]:
import sys
import os

sys.path.append(os.getcwd()[:-9])

import WickContractions

# the uglies part of the code is still defining operators.
from WickContractions.ops.operator import Operator
from WickContractions.ops.elemental import ElementalOperator
from WickContractions.ops.quarks import Quark
from WickContractions.ops.indexed import IndexedObject

In [2]:
# epsilion^abc (u_a Cg5 d_b^T) u_c
q1=Quark(False,'u','s_0','a','t_3','x_3')
g=IndexedObject('\\Gamma',['s_0','s_1'])
eps=IndexedObject('\\epsilon',['a','b','c'])
q2=Quark(False,'d','s_1','b','t_3','x_3')
q3=Quark(False,'u','s_2','c','t_3','x_3')

annihilate=Operator([ElementalOperator(+1,[g,eps],[q1,q2,q3])])


# epsilion^abc (baru_a Cg5 bard_b^T) baru_c
elementals=[]
for pi2 in [['u',+2/3],['d',-1/3]]:
    for pi1 in [['u',+2/3],['d',-1/3]]:
        q2=Quark(True,pi2[0],'s_3','c_1','t_2','x_2')
        g1=IndexedObject('\\gamma_{\mu}e^{-i\\mathbf{q}}',['s_3','s_4'])
        q3=Quark(False,pi2[0],'s_4','c_1','t_2','x_2')

        q4=Quark(True,pi1[0],'s_5','c_2','t_1','x_1')
        g2=IndexedObject('\\gamma_{\\nu}e^{i\\mathbf{q}}',['s_5','s_6'])
        q5=Quark(False,pi1[0],'s_6','c_2','t_1','x_1')
        
        pq1=Quark(True,'u','s_7','a\'','t_0','x_0')
        pg=IndexedObject('\\Gamma',['s_7','s_8'])
        peps=IndexedObject('\\epsilon',['a\'','b\'','c\''])
        pq2=Quark(True,'d','s_8','b\'','t_0','x_0')
        pq3=Quark(True,'u','s_9','c\'','t_0','x_0')
        
        elementals.append(ElementalOperator(-1*pi2[1]*pi1[1],[g1,g2,pg,peps],[q2,q3,q4,q5,pq1,pq2,pq3]))
        
create=Operator(elementals)

In [3]:
print(annihilate)

1\Gamma_{s_0 s_1}\epsilon_{a b c}u_{s_0 a}(x_3, t_3)d_{s_1 b}(x_3, t_3)u_{s_2 c}(x_3, t_3)


In [4]:
print(create)

-0.4444444444444444\gamma_{\mu}e^{-i\mathbf{q}}_{s_3 s_4}\gamma_{\nu}e^{i\mathbf{q}}_{s_5 s_6}\Gamma_{s_7 s_8}\epsilon_{a' b' c'}\bar{u}_{s_3 c_1}(x_2, t_2)u_{s_4 c_1}(x_2, t_2)\bar{u}_{s_5 c_2}(x_1, t_1)u_{s_6 c_2}(x_1, t_1)\bar{u}_{s_7 a'}(x_0, t_0)\bar{d}_{s_8 b'}(x_0, t_0)\bar{u}_{s_9 c'}(x_0, t_0)+0.2222222222222222\gamma_{\mu}e^{-i\mathbf{q}}_{s_3 s_4}\gamma_{\nu}e^{i\mathbf{q}}_{s_5 s_6}\Gamma_{s_7 s_8}\epsilon_{a' b' c'}\bar{u}_{s_3 c_1}(x_2, t_2)u_{s_4 c_1}(x_2, t_2)\bar{d}_{s_5 c_2}(x_1, t_1)d_{s_6 c_2}(x_1, t_1)\bar{u}_{s_7 a'}(x_0, t_0)\bar{d}_{s_8 b'}(x_0, t_0)\bar{u}_{s_9 c'}(x_0, t_0)+0.2222222222222222\gamma_{\mu}e^{-i\mathbf{q}}_{s_3 s_4}\gamma_{\nu}e^{i\mathbf{q}}_{s_5 s_6}\Gamma_{s_7 s_8}\epsilon_{a' b' c'}\bar{d}_{s_3 c_1}(x_2, t_2)d_{s_4 c_1}(x_2, t_2)\bar{u}_{s_5 c_2}(x_1, t_1)u_{s_6 c_2}(x_1, t_1)\bar{u}_{s_7 a'}(x_0, t_0)\bar{d}_{s_8 b'}(x_0, t_0)\bar{u}_{s_9 c'}(x_0, t_0)+-0.1111111111111111\gamma_{\mu}e^{-i\mathbf{q}}_{s_3 s_4}\gamma_{\nu}e^{i\mathbf{q}}_{s_5 

In [5]:
from WickContractions.wick.contract import contract
res=contract(annihilate,create)
len(res.diagrams)

60

In [6]:
import copy
from diagram_utils import combine_diagrams
total=copy.deepcopy(res.diagrams)

print(len(total))

##you can take the isospin limit by setting 'd'='u'
for d in total:
    for p in d.props:
        #p.name = 'S'
        p.name = 'S_'+p.name.split('_')[1]

combine_diagrams(total)

for d in total:
    d.coef*=9
        
print(len(total))

60
60


In [7]:
from print_utilities import diagram_as_latex_str, labelled_diagram_as_latex_str
from IPython.display import display, Math

In [8]:
print(diagram_as_latex_str(total[0],0))

d_{0}=4.0 S_{u}(x_3t_3\mid x_2t_2)_{\substack{s_0\\a}\substack{s_3\\c_1}}S_{u}(x_3t_3\mid x_1t_1)_{\substack{s_2\\c}\substack{s_5\\c_2}}S_{u}(x_2t_2\mid x_0t_0)_{\substack{s_4\\c_1}\substack{s_7\\a'}}S_{d}(x_3t_3\mid x_0t_0)_{\substack{s_1\\b}\substack{s_8\\b'}}S_{u}(x_1t_1\mid x_0t_0)_{\substack{s_6\\c_2}\substack{s_9\\c'}}['\\Gamma']['\\Gamma']['\\epsilon']['\\epsilon']\gamma_{{\mu}e^{-i\mathbf{q}}_{s_3s_4}}\gamma_{{\nu}e^{i\mathbf{q}}_{s_5s_6}}\\


In [9]:
#for i in range(0,len(total)):
#    display(Math(diagram_as_latex_str(total[i],i)))

In [10]:
from topologies_long import *

topList=diagram_time_order(total)
connect_times(topList)
name_topologies(topList)

In [11]:
topList

{0: [['2', '3'], ['1', '3'], ['0', '2'], ['0', '3'], ['0', '1']],
 1: [['2', '3'], ['1', '3'], ['0', '2'], ['0', '3'], ['0', '1']],
 2: [['2', '2'], ['0', '3'], ['0', '3'], ['0', '1'], ['1', '3']],
 3: [['2', '3'], ['1', '2'], ['0', '3'], ['0', '3'], ['0', '1']],
 4: [['2', '3'], ['1', '2'], ['0', '3'], ['0', '3'], ['0', '1']],
 5: [['2', '2'], ['0', '3'], ['0', '3'], ['0', '1'], ['1', '3']],
 6: [['2', '1'], ['1', '3'], ['0', '2'], ['0', '3'], ['0', '3']],
 7: [['1', '1'], ['0', '2'], ['0', '3'], ['0', '3'], ['2', '3']],
 8: [['2', '2'], ['0', '3'], ['0', '3'], ['0', '3'], ['1', '1']],
 9: [['2', '1'], ['1', '2'], ['0', '3'], ['0', '3'], ['0', '3']],
 10: [['2', '3'], ['1', '2'], ['0', '1'], ['0', '3'], ['0', '3']],
 11: [['2', '2'], ['0', '1'], ['0', '3'], ['0', '3'], ['1', '3']],
 12: [['2', '3'], ['1', '3'], ['0', '1'], ['0', '3'], ['0', '2']],
 13: [['2', '3'], ['1', '3'], ['0', '1'], ['0', '3'], ['0', '2']],
 14: [['2', '1'], ['1', '3'], ['0', '3'], ['0', '3'], ['0', '2']],
 15: 

In [12]:
x=copy.deepcopy(topList)
topSorted={k: v for k, v in sorted(x.items(), key=lambda item: item[1])}
print(topSorted)

{7: [['1', '1'], ['0', '2'], ['0', '3'], ['0', '3'], ['2', '3']], 18: [['1', '1'], ['0', '2'], ['0', '3'], ['0', '3'], ['2', '3']], 26: [['1', '1'], ['0', '2'], ['0', '3'], ['0', '3'], ['2', '3']], 32: [['1', '1'], ['0', '2'], ['0', '3'], ['0', '3'], ['2', '3']], 41: [['1', '1'], ['0', '3'], ['0', '2'], ['0', '3'], ['2', '3']], 47: [['1', '1'], ['0', '3'], ['0', '2'], ['0', '3'], ['2', '3']], 51: [['1', '1'], ['0', '3'], ['0', '2'], ['0', '3'], ['2', '3']], 58: [['1', '1'], ['0', '3'], ['0', '2'], ['0', '3'], ['2', '3']], 15: [['1', '1'], ['0', '3'], ['0', '3'], ['0', '2'], ['2', '3']], 16: [['1', '1'], ['0', '3'], ['0', '3'], ['0', '2'], ['2', '3']], 28: [['1', '1'], ['0', '3'], ['0', '3'], ['0', '2'], ['2', '3']], 29: [['1', '1'], ['0', '3'], ['0', '3'], ['0', '2'], ['2', '3']], 9: [['2', '1'], ['1', '2'], ['0', '3'], ['0', '3'], ['0', '3']], 22: [['2', '1'], ['1', '2'], ['0', '3'], ['0', '3'], ['0', '3']], 53: [['2', '1'], ['1', '2'], ['0', '3'], ['0', '3'], ['0', '3']], 54: [['2', 

In [13]:
from print_utilities import print_diagrams_labelled

#for i,label in topSorted.items():
    #display(Math(labelled_diagram_as_latex_str(total[i],i,label)))

In [14]:
tst = copy.deepcopy(total[0])

print(tst)

4.0 \Gamma_{s_0 s_1}\Gamma_{s_7 s_8}\epsilon_{a b c}\epsilon_{a' b' c'}\gamma_{\mu}e^{-i\mathbf{q}}_{s_3 s_4}\gamma_{\nu}e^{i\mathbf{q}}_{s_5 s_6}S_{u}(x_3,t_3\mid x_2,t_2)_{\substack{s_0\\a}\substack{s_3\\c_1}}S_{u}(x_3,t_3\mid x_1,t_1)_{\substack{s_2\\c}\substack{s_5\\c_2}}S_{u}(x_2,t_2\mid x_0,t_0)_{\substack{s_4\\c_1}\substack{s_7\\a'}}S_{d}(x_3,t_3\mid x_0,t_0)_{\substack{s_1\\b}\substack{s_8\\b'}}S_{u}(x_1,t_1\mid x_0,t_0)_{\substack{s_6\\c_2}\substack{s_9\\c'}}


In [15]:
from tracing import diagram_as_trace_string

for i,label in topSorted.items():
    display(Math('d_{{{}}}^{{\\text{{{}}}}}=\,'.format(i,label)+str(total[i])))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [16]:
print('\\beqs')
for i,label in topSorted.items():
    s='d_{{{}}}^{{\\text{{{}}}}}=\,'.format(i,label)+str(total[i])
    if(i!=len(topSorted.items())-1):
        s+='\\\\'
    print(s)
print('\\eeqs')

\beqs
d_{7}^{\text{[['1', '1'], ['0', '2'], ['0', '3'], ['0', '3'], ['2', '3']]}}=\,-4.0 \Gamma_{s_0 s_1}\Gamma_{s_7 s_8}\epsilon_{a b c}\epsilon_{a' b' c'}\gamma_{\mu}e^{-i\mathbf{q}}_{s_3 s_4}\gamma_{\nu}e^{i\mathbf{q}}_{s_5 s_6}S_{u}(x_3,t_3\mid x_2,t_2)_{\substack{s_0\\a}\substack{s_3\\c_1}}S_{u}(x_1,t_1\mid x_1,t_1)_{\substack{s_6\\c_2}\substack{s_5\\c_2}}S_{u}(x_2,t_2\mid x_0,t_0)_{\substack{s_4\\c_1}\substack{s_7\\a'}}S_{d}(x_3,t_3\mid x_0,t_0)_{\substack{s_1\\b}\substack{s_8\\b'}}S_{u}(x_3,t_3\mid x_0,t_0)_{\substack{s_2\\c}\substack{s_9\\c'}}\\
d_{18}^{\text{[['1', '1'], ['0', '2'], ['0', '3'], ['0', '3'], ['2', '3']]}}=\,4.0 \Gamma_{s_0 s_1}\Gamma_{s_7 s_8}\epsilon_{a b c}\epsilon_{a' b' c'}\gamma_{\mu}e^{-i\mathbf{q}}_{s_3 s_4}\gamma_{\nu}e^{i\mathbf{q}}_{s_5 s_6}S_{u}(x_3,t_3\mid x_2,t_2)_{\substack{s_2\\c}\substack{s_3\\c_1}}S_{u}(x_1,t_1\mid x_1,t_1)_{\substack{s_6\\c_2}\substack{s_5\\c_2}}S_{u}(x_2,t_2\mid x_0,t_0)_{\substack{s_4\\c_1}\substack{s_7\\a'}}S_{d}(x_3,t_3\mid